In [1]:
from agential.cog.lats.strategies.qa import LATSQAStrategy 
from agential.cog.lats.strategies.math import LATSMathStrategy 
from agential.cog.lats.strategies.code import LATSCodeStrategy 
from agential.llm.llm import LLM
from agential.cog.fewshots.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.fewshots.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.fewshots.humaneval import (
    HUMANEVAL_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.fewshots.mbpp import (
    MBPP_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,

    GSM8K_FEWSHOT_EXAMPLES_LATS_REFLECT,
    GSM8K_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_INSTRUCTION_GSM8K,
    LATS_REFLECT_INSTRUCTION_GSM8K,
    LATS_VALUE_INSTRUCTION_GSM8K,

    MBPP_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_MBPP,
    LATS_REFLECT_INSTRUCTION_MBPP,
    MBPP_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_VALUE_INSTRUCTION_MBPP,

    HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HUMANEVAL,
    LATS_REFLECT_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_VALUE_INSTRUCTION_HUMANEVAL,
)

from agential.cog.lats.functional import get_node_trajectory_qa, get_node_trajectory_math, get_node_trajectory_code


llm = LLM("gpt-3.5-turbo")

In [45]:
from agential.cog.lats.node import Node
from agential.cog.lats.output import LATSReActStepOutput
from agential.llm.llm import MockLLM

llm = MockLLM(
    "gpt-3.5-turbo",
    responses=["Explanation: Good trajectory. Correctness score: 8"],
)
strategy = LATSQAStrategy(llm=llm)

root = strategy.initialize()
child1 = Node(
    state=LATSReActStepOutput(
        thought="Child 1",
        action_type="",
        query="",
        observation="",
        answer="",
        external_tool_info={},
    ),
    parent=root,
)
child2 = Node(
    state=LATSReActStepOutput(
        thought="Child 2",
        action_type="",
        query="",
        observation="",
        answer="",
        external_tool_info={},
    ),
    parent=root,
    is_terminal=True,
)

root.children = [child1, child2]

question = "What is the capital of France?"
examples = "Example 1\nExample 2"
prompt = "Evaluate this trajectory"

strategy.reflection_map = [
    {
        "trajectory": "Failed trajectory",
        "reflection": "This trajectory failed because...",
    }
]

values, values_responses = strategy.evaluate_node(root, question, examples, prompt, {})


<PROMPT VALUE======================================================================>
Evaluate this trajectory
<PROMPT VALUE======================================================================>
<OUT VALUE======================================================================>
'Explanation: Good trajectory. Correctness score: 8'
<OUT VALUE======================================================================>


In [ ]:
for i in values_responses:
    print(i.choices[0].message.content)

In [ ]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

agent = LATSQAStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=2,
    reset=True
)

print(get_node_trajectory_qa(out.answer))

In [ ]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = -9867630

agent = LATSMathStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_GSM8K,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_GSM8K,
    value_prompt=LATS_VALUE_INSTRUCTION_GSM8K,            
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_math(out.answer))

In [3]:
import importlib
from agential.cog.lats.strategies import code

# Reload the module
importlib.reload(code)

# Now use the updated module
from agential.cog.lats.strategies.code import LATSCodeStrategy


In [4]:
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"

agent = LATSCodeStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HUMANEVAL,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HUMANEVAL,
    value_prompt=LATS_VALUE_INSTRUCTION_HUMANEVAL,            
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_code(out.answer))

IM HERE


AttributeError: 'tuple' object has no attribute 'answer'

: 

In [ ]:
out

In [ ]:
question = "Write a python function to find the first repeated character in a given string."
key = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""

agent = LATSCodeStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=MBPP_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=MBPP_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_MBPP,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_MBPP,
    value_prompt=LATS_VALUE_INSTRUCTION_MBPP,            
    additional_keys={"tests": key},
    reflect_additional_keys={"tests": key},
    value_additional_keys={"tests": key},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_code(out.answer))

In [ ]:
out